In [ ]:
!pip install pyspark
!unzip cvModel1.zip
!unzip pipelineFit.zip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=0ac105f8c300e49bc4d44387504a3bf1244f840d95eb207e5dc9bc5e04daed54
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
Archive:  cvModel1.zip
   creating: cvModel1/
   creating: cvModel1/bestModel/
   creating: cvModel1/bestModel/data/
 extracting: cvModel1/bestModel/data/.part-00000-e0e6fbd7-dd48-4880-82b5-6e2637cd0c07-c000.snappy.parquet.crc  
 extracting: cvModel1/bestModel/data/._SUCCESS.crc  
  inflating: cvModel1/bestModel/data/part-00000-e0e6fbd7-dd48-4880-82b5-6e2637cd0c07-c000.snappy.parquet  
 extracting: cvModel1/bestModel/data/_SUCCESS  
   creating: cvModel1/bestModel/metadata/
 extracting: cvModel1/bestModel/metadata/.part-00000.crc  
 extracting: cvModel1/bestModel/metadata/._SU

In [ ]:
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.sql.types import *
from pyspark.sql.functions import col
from pyspark.ml.feature import RegexTokenizer, CountVectorizer, Word2Vec, Tokenizer, StopWordsRemover
from pyspark.ml.classification import LogisticRegression, NaiveBayes, LinearSVC, OneVsRest, MultilayerPerceptronClassifier
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel
from pyspark.ml.pipeline import PipelineModel

In [ ]:
sc =SparkContext()
sqlContext = SQLContext(sc)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
customSchema = StructType([
    StructField("label", IntegerType()),
    StructField("text", StringType())])

In [ ]:
df_val = sqlContext.read.format("csv").option("header", "true").schema(customSchema).load('cleaned_twitter_validation.csv')

In [ ]:
regexTokenizer = RegexTokenizer(inputCol="text", outputCol="tokens", pattern=r" +")
tokenizer = Tokenizer(inputCol="text", outputCol="tokens")
countVectors = CountVectorizer(inputCol="tokens", outputCol="features", vocabSize=15000, minDF=5)
word2Vec = Word2Vec(vectorSize=100, minCount=0,maxIter=20, inputCol="tokens", outputCol="features")
hashingTF = HashingTF(inputCol="tokens", outputCol="rawFeatures", numFeatures=15000)
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=10) #minDocFreq: remove sparse terms

In [ ]:
# Fit the pipeline to validation documents.
pipeline = PipelineModel.load('pipelineFit')
dataset_val = pipeline.transform(df_val)
dataset_val.show(5)

+-----+--------------------+--------------------+--------------------+
|label|                text|              tokens|            features|
+-----+--------------------+--------------------+--------------------+
|    0|mentioned faceboo...|[mentioned, faceb...|(13363,[3,16,23,2...|
|    2|bbc news amazon b...|[bbc, news, amazo...|(13363,[2,34,138,...|
|    1|why pay  word  fu...|[why, pay, word, ...|(13363,[90,265,69...|
|    1|csgo matchmaking ...|[csgo, matchmakin...|(13363,[0,115,262...|
|    2|now  president sl...|[now, president, ...|(13363,[7,32,143,...|
+-----+--------------------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
cvModel = CrossValidatorModel.load('cvModel1')
predictions = cvModel.transform(dataset_val)

In [ ]:
predictions.filter(predictions['prediction'] == 0) \
    .select("text","probability","label","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 110)

+--------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------+-----+----------+
|                                                                                                          text|                                                                           probability|label|prediction|
+--------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------+-----+----------+
|words gratitude trevorphilipsstanaccount love   gta tumblr fandom particular   creating  talented  beautifu...|   [0.9988016127239734,8.530927674802259E-6,6.289466453168379E-4,5.609097030348915E-4]|    0|       0.0|
|the nigeria national team   ranked  29th best team  world  3rd africa  latest fifa world rankings  its  fir...| [0.9981446665691251

In [ ]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.9037828332690099